# Pressure Vessel Optimization

From: True Global Optimality of the Pressure Vessel Design Problem: A Benchmark for 
Bio-Inspired Optimisation Algorithms ( https://dx.doi.org/10.1504/IJBIC.2013.058910  or https://arxiv.org/abs/1403.7793)

**Global Optimum** (according to Yang): 6059.714335048436  @ [0.8125, 0.4375, 42.0984455958549, 176.6365958424394]

Note: In the runs below, you should be able to yield the exact global minimum, but in general, because a stochastic optimization method is being used, you will usually get close to, but not exactly to, the global minimum.  You may want to play with the control parameters of the optimization.

In [1]:
import numpy as np
from math import pi
import scipy.optimize
import PyCEGO

In [ ]:
# Allowed values for d1 and d2
alllowable_thicknesses = np.arange(0.0625, 999*0.0625+1, 0.0625)

def penalty_function(x):
    o = np.zeros_like(x)
    for ix, _x in enumerate(x):
        iR = ( _x < alllowable_thicknesses ).nonzero()[0][0]
        L,R = alllowable_thicknesses[iR-1],alllowable_thicknesses[iR]
        x01 = (_x-L)/(R-L)
        o[ix] = 1*0.5*(1-np.cos(2*np.pi*x01))
    return o

def meta_obj(x):
    return np.array([obj(_x) for _x in x])
  
def CEGO_obj(x):
    return obj([x[0].as_int()*0.0625,x[1].as_int()*0.0625,x[2].as_double(),x[3].as_double()])

def obj(x):
    d1,d2,r,L = x
    f = 0.6224*d1*r*L + 1.7781*r**2*d2 + 3.1661*L*d1**2 + 19.84*r*d1**2
    
    g = np.zeros((7,))
    g[1] = -d1 + 0.0193*r
    g[2] = -d2 + 0.00954*r
    g[3] = -pi*r**2*L - 4*pi/3*r**3 + 1296000
    g[4] = L - 240
    g[5],g[6] = penalty_function([d1,d2])
    
    s = np.ones_like(g)*100
    
    c = np.array([1 if g_i < 0 else 1+s_i*g_i for s_i,g_i in zip(s,g)])
    
    return f*np.product(c**3)

# True global minimum
true_minimum_Yang = 6059.714335048436
true_minimum_calc = obj([0.8125,0.4375,42.0984455958549,176.6365958424394])

print("the true minimum:",true_minimum_Yang,"@",[0.8125,0.4375,42.0984455958549,176.6365958424394])
assert(abs(true_minimum_Yang-true_minimum_calc) < 1e-2)

the true minimum: 6059.714335048436 @ [0.8125, 0.4375, 42.0984455958549, 176.6365958424394]


In [ ]:
D = 4
CEGO_bounds = [PyCEGO.Bound(1, 99),PyCEGO.Bound(1, 99),  # d1, d2
               PyCEGO.Bound(10.0,200.0),PyCEGO.Bound(10.0, 200.0) # r,L
               ]

for ocounter in range(5):
    
    layers = PyCEGO.NumberishLayers(CEGO_obj, D, D*20, 1, 3)    
    layers.set_bounds(CEGO_bounds)
    layers.set_builtin_evolver(PyCEGO.BuiltinEvolvers.differential_evolution)

    objs = []
    for counter in range(1000):
        layers.do_generation()
        objective, coeffs = layers.get_best()
        if counter % 50 == 0:
            print(layers.print_diagnostics())
        objs.append(objective)

    print('CEGO', objs[-1])

i: 0 best: 11701.5 c: 23, 16, 50.651392, 97.628987,  queue: 0
i: 50 best: 6319.91 c: 13, 7, 41.244917, 194.196254,  queue: 0
i: 100 best: 6192.75 c: 13, 7, 41.504769, 186.708521,  queue: 0
i: 150 best: 6063.78 c: 13, 7, 42.071433, 177.004533,  queue: 0
i: 200 best: 6059.74 c: 13, 7, 42.098419, 176.638012,  queue: 0
i: 250 best: 6059.71 c: 13, 7, 42.098443, 176.636635,  queue: 0
i: 300 best: 6059.71 c: 13, 7, 42.098446, 176.636596,  queue: 0
i: 350 best: 6059.71 c: 13, 7, 42.098446, 176.636596,  queue: 0
i: 400 best: 6059.71 c: 13, 7, 42.098446, 176.636596,  queue: 0
i: 450 best: 6059.71 c: 13, 7, 42.098446, 176.636596,  queue: 0
i: 500 best: 6059.71 c: 13, 7, 42.098446, 176.636596,  queue: 0
i: 550 best: 6059.71 c: 13, 7, 42.098446, 176.636596,  queue: 0
i: 600 best: 6059.71 c: 13, 7, 42.098446, 176.636596,  queue: 0
i: 650 best: 6059.71 c: 13, 7, 42.098446, 176.636596,  queue: 0
i: 700 best: 6059.71 c: 13, 7, 42.098446, 176.636596,  queue: 0
i: 750 best: 6059.71 c: 13, 7, 42.098446, 1